In [9]:
from os import makedirs
from os.path import join, isdir
import pytest
from numpy import array, linspace, ones, pi, zeros, sqrt, cos

from pyleecan.Classes.InputCurrent import InputCurrent
from pyleecan.Classes.MagElmer import MagElmer
from pyleecan.Classes.SlotM10 import SlotM10
from pyleecan.Classes.Simu1 import Simu1
from pyleecan.Classes.OPdq import OPdq
from pyleecan.Classes.Output import Output
from pyleecan.Functions.load import load
from pyleecan.Functions.Plot import dict_2D
from pyleecan.definitions import DATA_DIR
#save_plot_path="D:/KDH/pyleecan_out_figure/Result/"
#save_plot_path=DATA_DIR
#project_name = simu.mag.get_path_save_fea(output)
#save_plot_path="D:/KDH/pyleecan_out_figure/Result/"
# Gather results in the same folder
#save_path = join(save_plot_path, "Elmer")
#if not isdir(save_path):
#    makedirs(save_path)
from Tests import save_plot_path
save_path = join(save_plot_path, "Elmer")
if not isdir(save_path):
    makedirs(save_path)


mesh_dict = {
    "Lamination_Rotor_Bore_Radius_Ext": 180,
    "surface_line_0": 5,
    "surface_line_1": 10,
    "surface_line_2": 5,
    "surface_line_3": 5,
    "surface_line_4": 10,
    "surface_line_5": 5,
    "Lamination_Stator_Bore_Radius_Int": 10,
    "Lamination_Stator_Yoke_Side_Right": 30,
    "Lamination_Stator_Yoke_Side_Left": 30,
    "int_airgap_arc": 120,
    "int_sb_arc": 120,
    "ext_airgap_arc": 120,
    "ext_sb_arc": 120,
    "airbox_line_1": 10,
    "airbox_line_2": 10,
    "airbox_arc": 20,
}


Toyota_Prius = load(join(DATA_DIR, "Machine", "Toyota_Prius.json"))
Toyota_Prius.stator.slot.H1 = 1e-3
simu = Simu1(name="test_ipm_Elmer", machine=Toyota_Prius)

# Definition of the enforced output of the electrical module
# N0 = 1500
# Is = ImportMatrixVal(value=array([[20, -10, -10],[20, -10, -10],[20, -10, -10]]))
# Ir = ImportMatrixVal(value=zeros((1, 28)))
# Nt_tot = 3
# Na_tot = 4096
# simu.input = InputCurrent(
#     Is=Is,
#     Ir=Ir,  # zero current for the rotor
#     N0=N0,
#     Nt_tot=Nt_tot,
#     Na_tot=Na_tot,
#     angle_rotor_initial=0.2244,
# )

# Definition of a sinusoidal current
simu.input = InputCurrent()
# simu.input.Id_ref = 0  # [A]
# simu.input.Iq_ref = 250  # [A]
# simu.input.Nt_tot = 32 * 8    # Number of time step
# simu.input.Na_tot = 2048     # Spatial discretization
simu.input.OP = OPdq(N0=2000)  # Rotor speed [rpm]
p = Toyota_Prius.stator.winding.p
time = linspace(0, 60 / simu.input.OP.N0, num=32 * p, endpoint=False)
simu.input.time = time
simu.input.angle = linspace(0, 2 * pi, num=2048, endpoint=False)
I0 = 250
felec = p * simu.input.OP.N0 / 60
rot_dir = simu.machine.stator.comp_mmf_dir()
Phi0 = 140 * pi / 180
Ia = I0 * cos(2 * pi * felec * time + 0 * rot_dir * 2 * pi / 3 + Phi0)
Ib = I0 * cos(2 * pi * felec * time + 1 * rot_dir * 2 * pi / 3 + Phi0)
Ic = I0 * cos(2 * pi * felec * time + 2 * rot_dir * 2 * pi / 3 + Phi0)
# simu.input.set_Id_Iq(I0=250/sqrt(2), Phi0=140*pi/180)
simu.input.Is = array([Ia, Ib, Ic]).transpose()

# Definition of the magnetic simulation
# 2 sym + antiperiodicity = 1/4 Lamination
simu.mag = MagElmer(
    type_BH_stator=0,
    type_BH_rotor=0,
    is_periodicity_a=True,
    is_periodicity_t=True,
    FEA_dict=mesh_dict,
    is_get_mesh=True,
    is_save_FEA=True,
)
# Stop after magnetic computation
simu.force = None
simu.struct = None
# Run simulation
output = Output(simu=simu)
simu.run()

[23:43:34] Starting running simulation test_ipm_Elmer (machine=Toyota_Prius)
[23:43:34] Starting Magnetic module
[23:43:34] Calling ElmerGrid: C:\Program Files\Elmer 9.0-Release\bin\ElmerGrid.exe 14 2 C:\Users\KDH2018-PC\anaconda3\envs\py38_openturn\Lib\site-packages\pyleecan-1.3.7-py3.8.egg\pyleecan\Results\test_ipm_Elmer\Elmer\ELMER_simulation.msh -2d -autoclean -names -out C:\Users\KDH2018-PC\anaconda3\envs\py38_openturn\Lib\site-packages\pyleecan-1.3.7-py3.8.egg\pyleecan\Results\test_ipm_Elmer\Elmer\ELMER_simulation
[23:43:34] 
Starting program Elmergrid
Elmergrid reading in-line arguments
Lower dimensional boundaries will be removed
Materials and boundaries will be renumbered
Nodes that do not appear in any element will be removed
Output will be saved to file C:\Users\KDH2018-PC\anaconda3\envs\py38_openturn\Lib\site-packages\pyleecan-1.3.7-py3.8.

Elmergrid loading data:
-----------------------
Format chosen using the first line: $MeshFormat
Gmsh version is 4.1
Loading mesh in Gms

TypeError: cannot unpack non-iterable bool object

In [5]:
project_name = simu.mag.get_path_save_fea(output)
project_name

'C:\\Users\\KDH2018-PC\\anaconda3\\envs\\py38_openturn\\Lib\\site-packages\\pyleecan-1.3.7-py3.8.egg\\pyleecan\\Results\\test_ipm_Elmer\\Elmer\\ELMER_simulation'

In [46]:
from pyleecan.definitions import DATA_DIR
DATA_DIR

'C:/Users/KDH2018-PC/anaconda3/envs/py38_openturn/Lib/site-packages/pyleecan-1.3.7-py3.8.egg/pyleecan/Data'

gmsh

In [1]:
sym=4
from pyleecan.Functions.GMSH.draw_GMSH import draw_GMSH     
from pyleecan.Methods.Simulation.MagElmer import MagElmer_BP_dict


##comp_flux_airgap
#gmsh_filename = "D:/KDH/pyleecan_out_figure/Result/ELMER_simulation.msh"
project_name = simu.mag.get_path_save_fea(output)
gmsh_filename = project_name + ".msh"
gmsh_filename = project_name + ".msh"
elmermesh_folder = project_name
#gmsh_filename = self.get_path_save_fea(output) + ".msh"
output.mag.internal.FEA_dict = draw_GMSH(
    output=output,
    sym=sym,
    boundary_prop=MagElmer_BP_dict,
    is_lam_only_S=False,
    is_lam_only_R=False,
    user_mesh_dict=output.mag.internal.FEA_dict,
    is_sliding_band=True,
    is_airbox=True,
    path_save=gmsh_filename,
)
gmsh_filename

NameError: name 'simu' is not defined